In [ ]:
import os
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

from mapper_functions import plot_global_contour
from mapper_functions import plot_global

from my_functions import read_obsfcstana

In [ ]:
# Define the path directory
path_dir = '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg'

# Define the common file name start
file_name_start = 'ASCAT_new_scaling_innov.ens_avg.ldas_ObsFcstAna.'

# Define the print flag
printflag = False
    
# Initialize lists to store the mean values for each variable and the dates
obs_obs_mean_list_i = []
obs_fcst_mean_list_i = []
obs_ana_mean_list_i = []
omf_mean_list_i = []
oma_mean_list_i = []
dates_list_i = []
omf_max_list_i = []
omf_std_list_i = []
oma_max_list_i = []
oma_std_list_i = []

# Define the start and end dates
start_date = datetime.strptime('20150401', '%Y%m%d')
end_date = datetime.strptime('20210401', '%Y%m%d')

# Loop over the dates
current_date = start_date
while current_date <= end_date:
    # Define the file name for the current date
    file_name = file_name_start + current_date.strftime('%Y%m%d')
    if file_name[-2:] == '0101':
        print(file_name)
    
    # Call the read_obsfcstana function for the current file
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana(path_dir, file_name, printflag)

    # Convert the lists to numpy arrays
    obs_obs = np.array(obs_obs)
    obs_fcst = np.array(obs_fcst)
    obs_ana = np.array(obs_ana)
    
    # Calculate the mean values for the variables
    obs_obs_mean = np.mean(obs_obs)
    obs_fcst_mean = np.mean(obs_fcst)
    obs_ana_mean = np.mean(obs_ana)
    omf_mean = np.mean(obs_obs - obs_fcst)
    oma_mean = np.mean(obs_obs - obs_ana)
    
    # Calculate the maximum absolute difference between obs_obs and obs_fcst
    if obs_fcst.size > 0 and obs_obs.size > 0 and obs_fcst.shape == obs_obs.shape:
        omf_max = np.max(abs(obs_obs - obs_fcst))
        oma_max = np.max(abs(obs_obs - obs_ana))
    else:
        omf_max = np.nan
        print('Current date = ', current_date)
    omf_std = np.std(obs_obs - obs_fcst)
    oma_std = np.std(obs_obs - obs_ana)


    # Append the mean values to the lists
    obs_obs_mean_list_i.append(obs_obs_mean)
    obs_fcst_mean_list_i.append(obs_fcst_mean)
    obs_ana_mean_list_i.append(obs_ana_mean)
    omf_mean_list_i.append(omf_mean)
    oma_mean_list_i.append(oma_mean)
    omf_max_list_i.append(omf_max)
    omf_std_list_i.append(omf_std)
    oma_max_list_i.append(oma_max)
    oma_std_list_i.append(oma_std)
    
    # Append the current date to the dates list
    dates_list_i.append(current_date.strftime('%Y%m%d'))

    # Increment the current date by one day
    current_date += timedelta(days=1)

In [ ]:
# Create a bar plot of the mean values
plt.figure(figsize=(20,10))

# Calculate the mean of omf_mean_list_i
mean_value = np.nanmean(omf_mean_list_i)

bar_width = 0.6
opacity = 0.6
index = np.arange(len(dates_list_i))
plt.bar(index, omf_mean_list_i, bar_width, alpha=opacity, label='OmF mean - Innov')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Set the x-axis tick labels to show the first day of each month
xticks = [date for date in dates_list_i if date.endswith('0401')]
xtick_indices = [dates_list_i.index(date) for date in xticks]
plt.xticks(xtick_indices, xticks, rotation=45)

# Add a y label
plt.ylabel('SM (m$^3$ m$^{-3}$)')

# Add a title with two lines
plt.title(f'Mean global daily OmF - Innov\n(Mean Value: {mean_value:.6f})')

plt.legend()

# Save the figure as a PNG file
plt.savefig('OmF_mean_ts_i.png')

plt.show()

In [ ]:
# Create a bar plot of the mean values
plt.figure(figsize=(20,10))

# Calculate the mean of omf_mean_list_i
mean_value = np.nanmean(omf_std_list_i)

bar_width = 0.6
opacity = 0.6
index = np.arange(len(dates_list_i))
plt.bar(index, omf_std_list_i, bar_width, alpha=opacity, label='OmF mean - Innov')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Set the x-axis tick labels to show the first day of each month
xticks = [date for date in dates_list_i if date.endswith('0401')]
xtick_indices = [dates_list_i.index(date) for date in xticks]
plt.xticks(xtick_indices, xticks, rotation=45)

# Add a y label
plt.ylabel('SM (m$^3$ m$^{-3}$)')

# Add a title with two lines
plt.title(f'StdDev global daily OmF - Innov\n(Mean Value: {mean_value:.6f})')

plt.legend()

# Save the figure as a PNG file
plt.savefig('OmF_std_ts_i.png')

plt.show()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Create a DataFrame with dates and corresponding values
data = pd.DataFrame({'Date': dates_list_i[:-1], 'Value': obs_obs_mean_list_i[:-1]})

# Convert the 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Group the data by day of the year and calculate the mean value for each day
mean_values = data.groupby(data['Date'].dt.dayofyear)['Value'].transform('mean')

# Calculate the daily anomalies by subtracting the mean day of year value
data['Anomaly'] = data['Value'] - mean_values

# Create a DataFrame with dates and corresponding values
data2 = pd.DataFrame({'Date': dates_list_i[:-1], 'Value': obs_fcst_mean_list_i[:-1]})

# Convert the 'Date' column to datetime type
data2['Date'] = pd.to_datetime(data2['Date'])

# Group the data by day of the year and calculate the mean value for each day
mean_values = data2.groupby(data2['Date'].dt.dayofyear)['Value'].transform('mean')

# Calculate the daily anomalies by subtracting the mean day of year value
data2['Anomaly'] = data2['Value'] - mean_values

# Replace NaN values with the mean of the ten values around the NaN, excluding NaNs
data['Anomaly'] = data['Anomaly'].fillna(data['Anomaly'].rolling(21, min_periods=1, center=True).mean())
data2['Anomaly'] = data2['Anomaly'].fillna(data2['Anomaly'].rolling(21, min_periods=1, center=True).mean())

# Calculate the trend lines for data['Anomaly'] and data2['Anomaly']
data_trend = np.polyfit(data.index, data['Anomaly'], 1)
data2_trend = np.polyfit(data2.index, data2['Anomaly'], 1)

# Plot the whole time series of daily anomalies and the trend lines
plt.figure(figsize=(20, 10))
plt.plot(data['Date'], data['Anomaly'], label='Daily Obs Anomaly - Innov')
plt.plot(data2['Date'], data2['Anomaly'], label='Daily Fcst Anomaly - Innov')
plt.plot(data['Date'], np.polyval(data_trend, data.index), label=f'Trend in Obs: {data_trend[0]:.10f}')
plt.plot(data2['Date'], np.polyval(data2_trend, data2.index), label=f'Trend in Fcst: {data2_trend[0]:.10f}')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Add a y label
plt.ylabel('Daily Anomaly')

# Add a title
plt.title('Daily Anomalies - Innov')

plt.legend()

In [ ]:
# Create a DataFrame with dates and corresponding values
data = pd.DataFrame({'Date': dates_list_i[:-1], 'Value': obs_obs_mean_list_i[:-1]})

# Convert the 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Replace NaN values with the mean of the ten values around the NaN, excluding NaNs
data['Value'] = data['Value'].fillna(data['Value'].rolling(21, min_periods=1, center=True).mean())

# Calculate the mean of data['Value']
mean_value = data['Value'].mean()

# Calculate the anomaly by subtracting the mean from data['Value']
data['Anomaly'] = data['Value'] - mean_value

# Create a DataFrame with dates and corresponding values
data2 = pd.DataFrame({'Date': dates_list_i[:-1], 'Value': obs_fcst_mean_list_i[:-1]})

# Convert the 'Date' column to datetime type
data2['Date'] = pd.to_datetime(data2['Date'])

# Replace NaN values with the mean of the ten values around the NaN, excluding NaNs
data2['Value'] = data2['Value'].fillna(data2['Value'].rolling(21, min_periods=1, center=True).mean())

# Calculate the mean of data2['Value']
mean_value2 = data2['Value'].mean()

# Calculate the anomaly by subtracting the mean from data2['Value']
data2['Anomaly'] = data2['Value'] - mean_value2

# Calculate the trend lines for data['Anomaly'] and data2['Anomaly']
data_trend = np.polyfit(data.index, data['Anomaly'], 1)
data2_trend = np.polyfit(data2.index, data2['Anomaly'], 1)

# Plot obs_obs_mean_list_i against dates_list_i
plt.figure(figsize=(20,10))
plt.plot(data['Date'], data['Anomaly'], label='Obs')
plt.plot(data['Date'], np.polyval(data_trend, data.index), label=f'Trend in Obs: {data_trend[0]:.8f}')

# Plot obs_fcst_mean_list_i against dates_list_i
plt.plot(data2['Date'], data2['Anomaly'], label='Fcst')
plt.plot(data2['Date'], np.polyval(data2_trend, data2.index), label=f'Trend in Fcst: {data2_trend[0]:.8f}')

plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Add a y label
plt.ylabel('SM (m$^3$ m$^{-3}$)')

# Add a title
plt.title('Global mean daily Obs and Fcst anomaly')

plt.legend()
plt.show()

In [ ]:
# Create a DataFrame with dates and corresponding values
data = pd.DataFrame({'Date': dates_list_i[:-1], 'Value': obs_obs_mean_list_i[:-1]})

# Convert the 'Date' column to datetime type
data['Date'] = pd.to_datetime(data['Date'])

# Replace NaN values with the mean of the ten values around the NaN, excluding NaNs
data['Value'] = data['Value'].rolling(41, min_periods=1).mean()

# Calculate the mean of data['Value']
mean_value = data['Value'].mean()

# Calculate the anomaly by subtracting the mean from data['Value']
data['Anomaly'] = data['Value'] - mean_value

# Create a DataFrame with dates and corresponding values
data2 = pd.DataFrame({'Date': dates_list_i[:-1], 'Value': obs_fcst_mean_list_i[:-1]})

# Convert the 'Date' column to datetime type
data2['Date'] = pd.to_datetime(data2['Date'])

# Replace NaN values with the mean of the ten values around the NaN, excluding NaNs
data2['Value'] = data2['Value'].rolling(41, min_periods=1).mean()

# Calculate the mean of data2['Value']
mean_value2 = data2['Value'].mean()

# Calculate the anomaly by subtracting the mean from data2['Value']
data2['Anomaly'] = data2['Value'] - mean_value2

# Calculate the trend lines for data['Anomaly'] and data2['Anomaly']
data_trend = np.polyfit(data.index, data['Anomaly'], 1)
data2_trend = np.polyfit(data2.index, data2['Anomaly'], 1)

# Plot obs_obs_mean_list_i against dates_list_i
plt.figure(figsize=(20,10))
plt.plot(data['Date'], data['Anomaly'], label='Obs')
plt.plot(data['Date'], np.polyval(data_trend, data.index), label=f'Trend in Obs: {data_trend[0]:.8f}')

# Plot obs_fcst_mean_list_i against dates_list_i
plt.plot(data2['Date'], data2['Anomaly'], label='Fcst')
plt.plot(data2['Date'], np.polyval(data2_trend, data2.index), label=f'Trend in Fcst: {data2_trend[0]:.8f}')

plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Add a y label
plt.ylabel('SM (m$^3$ m$^{-3}$)')

# Add a title
plt.title('Global mean daily Obs and Fcst anomaly (40 day smoothing)')

plt.legend()
plt.show()

In [ ]:
# Define the path directory
path_dir = '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg'

# Define the common file name start
file_name_start = 'ASCAT_type_1_update_analy.ens_avg.ldas_ObsFcstAna.'

# Define the print flag
printflag = False

# Initialize lists to store the mean values for each variable and the dates
obs_obs_mean_list_a1 = []
obs_fcst_mean_list_a1 = []
obs_ana_mean_list_a1 = []
omf_mean_list_a1 = []
oma_mean_list_a1 = []
dates_list_a1 = []
omf_max_list_a1 = []
omf_std_list_a1 = []
oma_std_list_a1 = []

# Define the start and end dates
start_date = datetime.strptime('20150401', '%Y%m%d')
end_date = datetime.strptime('20160401', '%Y%m%d')

# Loop over the dates
current_date = start_date
while current_date <= end_date:
    # Define the file name for the current date
    file_name = file_name_start + current_date.strftime('%Y%m%d')
    if file_name[-2:] == '01':
        print(file_name)
    
    # Call the read_obsfcstana function for the current file
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana(path_dir, file_name, printflag)

    # Convert the lists to numpy arrays
    obs_obs = np.array(obs_obs)
    obs_fcst = np.array(obs_fcst)
    obs_ana = np.array(obs_ana)
    
    # Calculate the mean values for the variables
    obs_obs_mean = np.mean(obs_obs)
    obs_fcst_mean = np.mean(obs_fcst)
    obs_ana_mean = np.mean(obs_ana)
    omf_mean = np.mean(obs_obs - obs_fcst)
    oma_mean = np.mean(obs_obs - obs_ana)
    # Calculate the maximum absolute difference between obs_obs and obs_fcst
    if obs_fcst.size > 0 and obs_obs.size > 0 and obs_fcst.shape == obs_obs.shape:
        omf_max = np.max(abs(obs_obs - obs_fcst))
    else:
        omf_max = np.nan
        print('Current date = ', current_date)
    omf_std = np.std(obs_obs - obs_fcst)
    oma_std = np.std(obs_obs - obs_ana)


    # Append the mean values to the lists
    obs_obs_mean_list_a1.append(obs_obs_mean)
    obs_fcst_mean_list_a1.append(obs_fcst_mean)
    obs_ana_mean_list_a1.append(obs_ana_mean)
    omf_mean_list_a1.append(omf_mean)
    oma_mean_list_a1.append(oma_mean)
    omf_max_list_a1.append(omf_max)
    omf_std_list_a1.append(omf_std)
    oma_std_list_a1.append(oma_std)
    
    # Append the current date to the dates list
    dates_list_a1.append(current_date.strftime('%Y%m%d'))

    # Increment the current date by one day
    current_date += timedelta(days=1)

In [ ]:
# Define the path directory
path_dir = '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg'

# Define the common file name start
file_name_start = 'ASCAT_type_2_aggress_bugfix.ens_avg.ldas_ObsFcstAna.'

# Define the print flag
printflag = False

# Initialize lists to store the mean values for each variable and the dates
obs_obs_mean_list_a2 = []
obs_fcst_mean_list_a2 = []
obs_ana_mean_list_a2 = []
omf_mean_list_a2 = []
oma_mean_list_a2 = []
dates_list_a2 = []
omf_max_list_a2 = []
omf_std_list_a2 = []
oma_std_list_a2 = []

# Define the start and end dates
start_date = datetime.strptime('20150401', '%Y%m%d')
end_date = datetime.strptime('20160401', '%Y%m%d')

# Loop over the dates
current_date = start_date
while current_date <= end_date:
    # Define the file name for the current date
    file_name = file_name_start + current_date.strftime('%Y%m%d')
    if file_name[-2:] == '01':
        print(file_name)
    
    # Call the read_obsfcstana function for the current file
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana(path_dir, file_name, printflag)

    # Convert the lists to numpy arrays
    obs_obs = np.array(obs_obs)
    obs_fcst = np.array(obs_fcst)
    obs_ana = np.array(obs_ana)
    
    # Calculate the mean values for the variables
    obs_obs_mean = np.mean(obs_obs)
    obs_fcst_mean = np.mean(obs_fcst)
    obs_ana_mean = np.mean(obs_ana)
    omf_mean = np.mean(obs_obs - obs_fcst)
    oma_mean = np.mean(obs_obs - obs_ana)
    # Calculate the maximum absolute difference between obs_obs and obs_fcst
    if obs_fcst.size > 0 and obs_obs.size > 0 and obs_fcst.shape == obs_obs.shape:
        omf_max = np.max(abs(obs_obs - obs_fcst))
    else:
        omf_max = np.nan
        print('Current date = ', current_date)
    omf_std = np.std(obs_obs - obs_fcst)
    oma_std = np.std(obs_obs - obs_ana)


    # Append the mean values to the lists
    obs_obs_mean_list_a2.append(obs_obs_mean)
    obs_fcst_mean_list_a2.append(obs_fcst_mean)
    obs_ana_mean_list_a2.append(obs_ana_mean)
    omf_mean_list_a2.append(omf_mean)
    oma_mean_list_a2.append(oma_mean)
    omf_max_list_a2.append(omf_max)
    omf_std_list_a2.append(omf_std)
    oma_std_list_a2.append(oma_std)
    
    # Append the current date to the dates list
    dates_list_a2.append(current_date.strftime('%Y%m%d'))

    # Increment the current date by one day
    current_date += timedelta(days=1)

In [ ]:
# Create a bar plot of the mean values
plt.figure(figsize=(20,10))
bar_width = 0.6
opacity = 0.6
index = np.arange(len(dates_list_a1))
plt.bar(index, omf_mean_list_i[:len(index)], bar_width, alpha=opacity, label='OmF mean - Innov')
plt.bar(index, omf_mean_list_a1, bar_width, alpha=opacity, label='OmF mean - Ana_1')
plt.bar(index, omf_mean_list_a2, bar_width, alpha=opacity, label='OmF mean - Ana_2')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Set the x-axis tick labels to show the first day of each month
xticks = [date for date in dates_list_a1 if date.endswith('01')]
xtick_indices = [dates_list_a1.index(date) for date in xticks]
plt.xticks(xtick_indices, xticks, rotation=45)

# Add a y label
plt.ylabel('SM (m$^3$ m$^{-3}$)')

plt.legend()

# Add a title
plt.title('Mean global daily OmF for innovation run, update type 1 and update type 2 analysis')

# Save the figure as a PNG file
plt.savefig('OmF_mean_ts_i_a.png')

plt.show()

In [ ]:
# Create a bar plot of the mean values
plt.figure(figsize=(20,10))
bar_width = 0.6
opacity = 0.6
index = np.arange(len(dates_list_a1))
plt.bar(index, omf_std_list_i[:len(index)], bar_width, alpha=opacity, label='OmF StdDev - Innov')
plt.bar(index, omf_std_list_a1, bar_width, alpha=opacity, label='OmF StdDev - Ana_1')
plt.bar(index, omf_std_list_a2, bar_width, alpha=opacity, label='OmF StdDev - Ana_2')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Set the x-axis tick labels to show the first day of each month
xticks = [date for date in dates_list_a1 if date.endswith('01')]
xtick_indices = [dates_list_a1.index(date) for date in xticks]
plt.xticks(xtick_indices, xticks, rotation=45)

# Add a y label
plt.ylabel('SM (m$^3$ m$^{-3}$)')

plt.legend()

# Add a title
plt.title('Stddev of global daily OmF for innovation run, update type 1 and update type 2 analysis')

# Save the figure as a PNG file
plt.savefig('OmF_std_ts_i_a.png')

plt.show()

In [ ]:
# Create a bar plot of the mean values
plt.figure(figsize=(20,10))
bar_width = 0.6
opacity = 0.6
index = np.arange(len(dates_list_a1))
plt.bar(index, omf_mean_list_a2, bar_width, alpha=opacity, label='OmF mean - Ana_2')
plt.bar(index, oma_mean_list_a2, bar_width, alpha=opacity, label='OmA mean - Ana_2')
plt.axhline(y=0, color='black', linestyle='--', linewidth=0.5) # Add a black line at y=0

# Set the x-axis tick labels to show the first day of each month
xticks = [date for date in dates_list_a1 if date.endswith('01')]
xtick_indices = [dates_list_a1.index(date) for date in xticks]
plt.xticks(xtick_indices, xticks, rotation=45)

# Add a y label
plt.ylabel('SM (m$^3$ m$^{-3}$)')

plt.legend()

# Add a title
plt.title('Mean global daily OmF and OmA for update type 2 analysis')

# Save the figure as a PNG file
plt.savefig('OmF_OmA_mean_ts_a.png')

plt.show()

In [ ]:
# Define the list of paths
paths = ['/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M04',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M05',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M06',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M07',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M08',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M09',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M10',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M11',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M12',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2016/M01',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2016/M02',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_1_update_analy/ASCAT_type_1_update_analy/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2016/M03']
         

# Define the common file name start
file_name_start = 'ASCAT_type_1_update_analy.ens_avg.ldas_ObsFcstAna.20'

# Define the print flag
printflag = False

# Initialize lists to store the returned values
date_times = []
obs_species_list = []
obs_tilenum_list = []
obs_lon_list = []
obs_lat_list = []
obs_obs_list = []
obs_fcst_list = []
obs_ana_list = []


# Loop over the paths
for path in paths:
    # Call the read_obsfcstana function for the current path
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana(path, file_name_start, printflag)
    
    # Append the returned values to the lists
    date_times.append(date_time)
    obs_species_list.append(obs_species)
    obs_tilenum_list.append(obs_tilenum)
    obs_lon_list.append(obs_lon)
    obs_lat_list.append(obs_lat)
    obs_obs_list.append(obs_obs)
    obs_fcst_list.append(obs_fcst)
    obs_ana_list.append(obs_ana)

# Combine the returned values from all paths
date_time = np.concatenate(date_times)
obs_species = np.concatenate(obs_species_list)
obs_tilenum = np.concatenate(obs_tilenum_list)
obs_lon = np.concatenate(obs_lon_list)
obs_lat = np.concatenate(obs_lat_list)
obs_obs = np.concatenate(obs_obs_list)
obs_fcst = np.concatenate(obs_fcst_list)
obs_ana = np.concatenate(obs_ana_list)

nodata = -9999

obs_obs[obs_obs == nodata] = np.nan
obs_fcst[obs_fcst == nodata] = np.nan
obs_ana[obs_ana == nodata] = np.nan

In [ ]:
# Put the observation data into a 2D array

#Calculate i,j
ll_lon = -180.0
ll_lat = -90.0
dlon = 0.25
dlat = 0.25

# Calculate the number of grid cells in the x direction
nx = int((180.0 - ll_lon) / dlon)

# Calculate the number of grid cells in the y direction
ny = int((90.0 - ll_lat) / dlat)

# Calculate lon and lat
lon = np.linspace(ll_lon, ll_lon + nx * dlon, nx)
lat = np.linspace(ll_lat, ll_lat + ny * dlat, ny)

# Create 2D grid of lons and lats
lon2d, lat2d = np.meshgrid(lon, lat)

n_obs = len(obs_obs)

i_out = np.zeros(n_obs)
j_out = np.zeros(n_obs)

for i in range(n_obs):
    i_out[i] = np.floor((obs_lon[i] - ll_lon)/dlon)
    j_out[i] = np.floor((obs_lat[i] - ll_lat)/dlat)

# Initialize a list of lists to store the observations for each grid cell
obs_list = [[[] for i in range(ny)] for j in range(nx)]
fcst_list = [[[] for i in range(ny)] for j in range(nx)]
omf_list = [[[] for i in range(ny)] for j in range(nx)]
oma_list = [[[] for i in range(ny)] for j in range(nx)]
inc_list = [[[] for i in range(ny)] for j in range(nx)]

# Find all the observations that fall into each grid cell
for i in range(n_obs):
    obs_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i])
    fcst_list[int(i_out[i])][int(j_out[i])].append(obs_fcst[i])
    omf_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i] - obs_fcst[i])
    oma_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i] - obs_ana[i])
    inc_list[int(i_out[i])][int(j_out[i])].append(obs_ana[i] - obs_fcst[i])

# Calculate the mean of the observations in each grid cell
obs_mean_a1 = np.zeros([nx, ny])
obs_count_a1 = np.zeros([nx, ny])
fcst_mean_a1 = np.zeros([nx, ny])
omf_mean_a1 = np.zeros([nx, ny])
omf_max_a1 = np.zeros([nx, ny])
omf_std_a1 = np.zeros([nx, ny])
oma_mean_a1 = np.zeros([nx, ny])
oma_max_a1 = np.zeros([nx, ny])
oma_std_a1 = np.zeros([nx, ny])
inc_mean_a1 = np.zeros([nx, ny])
inc_max_a1 = np.zeros([nx, ny])
inc_std_a1 = np.zeros([nx, ny])

for i in range(nx):
    for j in range(ny):
        if len(obs_list[i][j]) > 0:
            obs_mean_a1[i, j] = np.nanmean(obs_list[i][j])
            obs_count_a1[i, j] = len(obs_list[i][j])
            fcst_mean_a1[i, j] = np.nanmean(fcst_list[i][j])
            omf_mean_a1[i, j] = np.nanmean(omf_list[i][j])
            omf_max_a1[i, j] = np.nanmax(abs(np.array(omf_list[i][j])))
            omf_std_a1[i, j] = np.nanstd(omf_list[i][j])
            oma_mean_a1[i, j] = np.nanmean(oma_list[i][j])
            oma_max_a1[i, j] = np.nanmax(abs(np.array(oma_list[i][j])))
            oma_std_a1[i, j] = np.nanstd(oma_list[i][j])
            inc_mean_a1[i, j] = np.nanmean(inc_list[i][j])
            inc_max_a1[i, j] = np.nanmax(abs(np.array(inc_list[i][j])))
            inc_std_a1[i, j] = np.nanstd(inc_list[i][j])
        else:
            obs_mean_a1[i, j] = np.nan
            obs_count_a1[i, j] = np.nan
            fcst_mean_a1[i, j] = np.nan
            omf_mean_a1[i, j] = np.nan
            omf_max_a1[i, j] = np.nan
            omf_std_a1[i, j] = np.nan
            oma_mean_a1[i, j] = np.nan
            oma_max_a1[i, j] = np.nan
            oma_std_a1[i, j] = np.nan
            inc_mean_a1[i, j] = np.nan
            inc_max_a1[i, j] = np.nan
            inc_std_a1[i, j] = np.nan
            

In [ ]:
# Define the list of paths
paths = ['/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M04',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M05',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M06',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M07',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M08',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M09',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M10',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M11',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M12',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2016/M01',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2016/M02',
         '/discover/nobackup/amfox/Experiments/ASCAT_new_scaling_innov/ASCAT_new_scaling_innov/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2016/M03']

# Define the common file name start
file_name_start = 'ASCAT_new_scaling_innov.ens_avg.ldas_ObsFcstAna.20'

# Define the print flag
printflag = False

# Initialize lists to store the returned values
date_times = []
obs_species_list = []
obs_tilenum_list = []
obs_lon_list = []
obs_lat_list = []
obs_obs_list = []
obs_fcst_list = []
obs_ana_list = []

# Loop over the paths
for path in paths:
    # Call the read_obsfcstana function for the current path
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana(path, file_name_start, printflag)
    
    # Append the returned values to the lists
    date_times.append(date_time)
    obs_species_list.append(obs_species)
    obs_tilenum_list.append(obs_tilenum)
    obs_lon_list.append(obs_lon)
    obs_lat_list.append(obs_lat)
    obs_obs_list.append(obs_obs)
    obs_fcst_list.append(obs_fcst)
    obs_ana_list.append(obs_ana)

# Combine the returned values from all paths
date_time = np.concatenate(date_times)
obs_species = np.concatenate(obs_species_list)
obs_tilenum = np.concatenate(obs_tilenum_list)
obs_lon = np.concatenate(obs_lon_list)
obs_lat = np.concatenate(obs_lat_list)
obs_obs = np.concatenate(obs_obs_list)
obs_fcst = np.concatenate(obs_fcst_list)
obs_ana = np.concatenate(obs_ana_list)

nodata = -9999

obs_obs[obs_obs == nodata] = np.nan
obs_fcst[obs_fcst == nodata] = np.nan
obs_ana[obs_ana == nodata] = np.nan

In [ ]:
# Put the observation data into a 2D array

#Calculate i,j
ll_lon = -180.0
ll_lat = -90.0
dlon = 0.25
dlat = 0.25

# Calculate the number of grid cells in the x direction
nx = int((180.0 - ll_lon) / dlon)

# Calculate the number of grid cells in the y direction
ny = int((90.0 - ll_lat) / dlat)

# Calculate lon and lat
lon = np.linspace(ll_lon, ll_lon + nx * dlon, nx)
lat = np.linspace(ll_lat, ll_lat + ny * dlat, ny)

# Create 2D grid of lons and lats
lon2d, lat2d = np.meshgrid(lon, lat)

n_obs = len(obs_obs)

i_out = np.zeros(n_obs)
j_out = np.zeros(n_obs)

for i in range(n_obs):
    i_out[i] = np.floor((obs_lon[i] - ll_lon)/dlon)
    j_out[i] = np.floor((obs_lat[i] - ll_lat)/dlat)

# Initialize a list of lists to store the observations for each grid cell
obs_list = [[[] for i in range(ny)] for j in range(nx)]
fcst_list = [[[] for i in range(ny)] for j in range(nx)]
omf_list = [[[] for i in range(ny)] for j in range(nx)]
oma_list = [[[] for i in range(ny)] for j in range(nx)]
inc_list = [[[] for i in range(ny)] for j in range(nx)]

# Find all the observations that fall into each grid cell
for i in range(n_obs):
    obs_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i])
    fcst_list[int(i_out[i])][int(j_out[i])].append(obs_fcst[i])
    omf_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i] - obs_fcst[i])
    oma_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i] - obs_ana[i])
    inc_list[int(i_out[i])][int(j_out[i])].append(obs_ana[i] - obs_fcst[i])

# Calculate the mean of the observations in each grid cell
obs_mean_i = np.zeros([nx, ny])
obs_count_i = np.zeros([nx, ny])
fcst_mean_i = np.zeros([nx, ny])
omf_mean_i = np.zeros([nx, ny])
omf_max_i = np.zeros([nx, ny])
omf_std_i = np.zeros([nx, ny])
oma_mean_i = np.zeros([nx, ny])
oma_max_i = np.zeros([nx, ny])
oma_std_i = np.zeros([nx, ny])
inc_mean_i = np.zeros([nx, ny])
inc_max_i = np.zeros([nx, ny])
inc_std_i = np.zeros([nx, ny])

for i in range(nx):
    for j in range(ny):
        if len(obs_list[i][j]) > 0:
            obs_mean_i[i, j] = np.nanmean(obs_list[i][j])
            obs_count_i[i, j] = len(obs_list[i][j])
            fcst_mean_i[i, j] = np.nanmean(fcst_list[i][j])
            omf_mean_i[i, j] = np.nanmean(omf_list[i][j])
            omf_max_i[i, j] = np.nanmax(abs(np.array(omf_list[i][j])))
            omf_std_i[i, j] = np.nanstd(omf_list[i][j])
            oma_mean_i[i, j] = np.nanmean(oma_list[i][j])
            oma_max_i[i, j] = np.nanmax(abs(np.array(oma_list[i][j])))
            oma_std_i[i, j] = np.nanstd(oma_list[i][j])
            inc_mean_i[i, j] = np.nanmean(inc_list[i][j])
            inc_max_i[i, j] = np.nanmax(abs(np.array(inc_list[i][j])))
            inc_std_i[i, j] = np.nanstd(inc_list[i][j])
        else:
            obs_mean_i[i, j] = np.nan
            obs_count_i[i, j] = np.nan
            fcst_mean_i[i, j] = np.nan
            omf_mean_i[i, j] = np.nan
            omf_max_i[i, j] = np.nan
            omf_std_i[i, j] = np.nan
            oma_mean_i[i, j] = np.nan
            oma_max_i[i, j] = np.nan
            oma_std_i[i, j] = np.nan
            inc_mean_i[i, j] = np.nan
            inc_max_i[i, j] = np.nan
            inc_std_i[i, j] = np.nan

In [ ]:
print(len(obs_obs))
print(len(obs_ana))

In [ ]:
# Define the list of paths
paths = ['/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M04',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M05',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M06',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M07',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M08',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M09',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M10',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M11',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M12',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M01',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M02',
         '/discover/nobackup/amfox/Experiments/ASCAT_type_2_aggress_bugfix/ASCAT_type_2_aggress_bugfix/output/SMAP_EASEv2_M36_GLOBAL/ana/ens_avg/Y2015/M03']

# Define the common file name start
file_name_start = 'ASCAT_type_2_aggress_bugfix.ens_avg.ldas_ObsFcstAna.20'

# Define the print flag
printflag = False

# Initialize lists to store the returned values
date_times = []
obs_species_list = []
obs_tilenum_list = []
obs_lon_list = []
obs_lat_list = []
obs_obs_list = []
obs_fcst_list = []
obs_ana_list = []


# Loop over the paths
for path in paths:
    # Call the read_obsfcstana function for the current path
    date_time, obs_species, obs_tilenum, obs_lon, obs_lat, obs_obs, obs_obsvar, obs_fcst, obs_fcstvar, obs_ana, obs_anavar = read_obsfcstana(path, file_name_start, printflag)
    
    # Append the returned values to the lists
    date_times.append(date_time)
    obs_species_list.append(obs_species)
    obs_tilenum_list.append(obs_tilenum)
    obs_lon_list.append(obs_lon)
    obs_lat_list.append(obs_lat)
    obs_obs_list.append(obs_obs)
    obs_fcst_list.append(obs_fcst)
    obs_ana_list.append(obs_ana)

# Combine the returned values from all paths
date_time = np.concatenate(date_times)
obs_species = np.concatenate(obs_species_list)
obs_tilenum = np.concatenate(obs_tilenum_list)
obs_lon = np.concatenate(obs_lon_list)
obs_lat = np.concatenate(obs_lat_list)
obs_obs = np.concatenate(obs_obs_list)
obs_fcst = np.concatenate(obs_fcst_list)
obs_ana = np.concatenate(obs_ana_list)

nodata = -9999

obs_obs[obs_obs == nodata] = np.nan
obs_fcst[obs_fcst == nodata] = np.nan
obs_ana[obs_ana == nodata] = np.nan

In [ ]:
# Put the observation data into a 2D array

#Calculate i,j
ll_lon = -180.0
ll_lat = -90.0
dlon = 0.25
dlat = 0.25

# Calculate the number of grid cells in the x direction
nx = int((180.0 - ll_lon) / dlon)

# Calculate the number of grid cells in the y direction
ny = int((90.0 - ll_lat) / dlat)

# Calculate lon and lat
lon = np.linspace(ll_lon, ll_lon + nx * dlon, nx)
lat = np.linspace(ll_lat, ll_lat + ny * dlat, ny)

# Create 2D grid of lons and lats
lon2d, lat2d = np.meshgrid(lon, lat)

n_obs = len(obs_obs)

i_out = np.zeros(n_obs)
j_out = np.zeros(n_obs)

for i in range(n_obs):
    i_out[i] = np.floor((obs_lon[i] - ll_lon)/dlon)
    j_out[i] = np.floor((obs_lat[i] - ll_lat)/dlat)

# Initialize a list of lists to store the observations for each grid cell
obs_list = [[[] for i in range(ny)] for j in range(nx)]
fcst_list = [[[] for i in range(ny)] for j in range(nx)]
omf_list = [[[] for i in range(ny)] for j in range(nx)]
oma_list = [[[] for i in range(ny)] for j in range(nx)]
inc_list = [[[] for i in range(ny)] for j in range(nx)]

# Find all the observations that fall into each grid cell
for i in range(n_obs):
    obs_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i])
    fcst_list[int(i_out[i])][int(j_out[i])].append(obs_fcst[i])
    omf_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i] - obs_fcst[i])
    oma_list[int(i_out[i])][int(j_out[i])].append(obs_obs[i] - obs_ana[i])
    inc_list[int(i_out[i])][int(j_out[i])].append(obs_ana[i] - obs_fcst[i])

# Calculate the mean of the observations in each grid cell
obs_mean_a2 = np.zeros([nx, ny])
obs_count_a2 = np.zeros([nx, ny])
fcst_mean_a2 = np.zeros([nx, ny])
omf_mean_a2 = np.zeros([nx, ny])
omf_max_a2 = np.zeros([nx, ny])
omf_std_a2 = np.zeros([nx, ny])
oma_mean_a2 = np.zeros([nx, ny])
oma_max_a2 = np.zeros([nx, ny])
oma_std_a2 = np.zeros([nx, ny])
inc_mean_a2 = np.zeros([nx, ny])
inc_max_a2 = np.zeros([nx, ny])
inc_std_a2 = np.zeros([nx, ny])

for i in range(nx):
    for j in range(ny):
        if len(obs_list[i][j]) > 0:
            obs_mean_a2[i, j] = np.nanmean(obs_list[i][j])
            obs_count_a2[i, j] = len(obs_list[i][j])
            fcst_mean_a2[i, j] = np.nanmean(fcst_list[i][j])
            omf_mean_a2[i, j] = np.nanmean(omf_list[i][j])
            omf_max_a2[i, j] = np.nanmax(abs(np.array(omf_list[i][j])))
            omf_std_a2[i, j] = np.nanstd(omf_list[i][j])
            oma_mean_a2[i, j] = np.nanmean(oma_list[i][j])
            oma_max_a2[i, j] = np.nanmax(abs(np.array(oma_list[i][j])))
            oma_std_a2[i, j] = np.nanstd(oma_list[i][j])
            inc_mean_a2[i, j] = np.nanmean(inc_list[i][j])
            inc_max_a2[i, j] = np.nanmax(abs(np.array(inc_list[i][j])))
            inc_std_a2[i, j] = np.nanstd(inc_list[i][j])
        else:
            obs_mean_a2[i, j] = np.nan
            obs_count_a2[i, j] = np.nan
            fcst_mean_a2[i, j] = np.nan
            omf_mean_a2[i, j] = np.nan
            omf_max_a2[i, j] = np.nan
            omf_std_a2[i, j] = np.nan
            oma_mean_a2[i, j] = np.nan
            oma_max_a2[i, j] = np.nan
            oma_std_a2[i, j] = np.nan
            inc_mean_a2[i, j] = np.nan
            inc_max_a2[i, j] = np.nan
            inc_std_a2[i, j] = np.nan

In [ ]:
plot_global_contour(lon2d, lat2d, fcst_mean_i.T, False, 'Fcst mean (04/2105 - 03/2016) - Innovation', '(m3 m3-1)',0,1.0)

In [ ]:
plot_global_contour(lon2d, lat2d, fcst_mean_a2.T, False, 'Fcst mean (04/2105 - 03/2016)- Analysis', '(m3 m3-1)',0,1.0)

In [ ]:
plot_global_contour(lon2d, lat2d, (fcst_mean_a2 - fcst_mean_i).T, False, 'Fcst mean (04/2105 - 03/2016) - Diff (Analy - Innov)', '(m3 m3-1)', -0.15, 0.15)

In [ ]:
plot_global_contour(lon2d, lat2d, omf_mean_i.T, False, 'OmF mean (04/2105 - 03/2016) - Innovation', '(m3 m3-1)')

In [ ]:
plot_global_contour(lon2d, lat2d, omf_mean_a2.T, False, 'OmF mean (04/2105 - 03/2016)- Analysis', '(m3 m3-1)')

In [ ]:
plot_global_contour(lon2d, lat2d, (omf_mean_a2 - omf_mean_i).T, False, 'OmF mean (04/2105 - 03/2016) - Diff (Analy - Innov)', '(m3 m3-1)')

In [ ]:
plot_global_contour(lon2d, lat2d, omf_std_i.T, False, 'OmF stddev (04/2105 - 03/2016) - Innovation', '(m3 m3-1)')

In [ ]:
plot_global_contour(lon2d, lat2d, omf_std_a2.T, False, 'OmF stddev (04/2105 - 03/2016)- Analysis', '(m3 m3-1)')

In [ ]:
plot_global_contour(lon2d, lat2d, (omf_std_a2 - omf_std_i).T, False, 'stddev mean (04/2105 - 03/2016) - Diff (Analy - Innov)', '(m3 m3-1)', -0.05, 0.05)